In [2]:
import pandas as pd

# read in csvs
daily_calories = pd.read_csv("https://raw.githubusercontent.com/qzhang21/BMI706_FinalProject/main/dailyCalories_merged.csv")
daily_steps = pd.read_csv("https://raw.githubusercontent.com/qzhang21/BMI706_FinalProject/main/dailySteps_merged.csv")

print(daily_calories.head())
print(daily_steps.head())




           Id ActivityDay  Calories
0  1503960366   4/12/2016      1985
1  1503960366   4/13/2016      1797
2  1503960366   4/14/2016      1776
3  1503960366   4/15/2016      1745
4  1503960366   4/16/2016      1863
           Id ActivityDay  StepTotal
0  1503960366   4/12/2016      13162
1  1503960366   4/13/2016      10735
2  1503960366   4/14/2016      10460
3  1503960366   4/15/2016       9762
4  1503960366   4/16/2016      12669


In [9]:
daily_steps

,Id,ActivityDay,StepTotal
0,1503960366,4/12/2016,13162
1,1503960366,4/13/2016,10735
2,1503960366,4/14/2016,10460
3,1503960366,4/15/2016,9762
4,1503960366,4/16/2016,12669
...,...,...,...
935,8877689391,5/8/2016,10686
936,8877689391,5/9/2016,20226
937,8877689391,5/10/2016,10733
938,8877689391,5/11/2016,21420


In [8]:
import numpy as np
np.unique(daily_calories['Id'])
daily_calories.merge(daily_steps, on=["Id", "ActivityDay"]) # merge files

,Id,ActivityDay,Calories,StepTotal
0,1503960366,4/12/2016,1985,13162
1,1503960366,4/13/2016,1797,10735
2,1503960366,4/14/2016,1776,10460
3,1503960366,4/15/2016,1745,9762
4,1503960366,4/16/2016,1863,12669
...,...,...,...,...
935,8877689391,5/8/2016,2847,10686
936,8877689391,5/9/2016,3710,20226
937,8877689391,5/10/2016,2832,10733
938,8877689391,5/11/2016,3832,21420


In [ ]:
import streamlit as st
import altair as alt

chart = alt.Chart(subset).mark_rect().encode(
    x=alt.X("Age", sort=ages),
    y=alt.Y('Country'),
    color=alt.Color("Rate", scale=alt.Scale(type='log', domain=(0.01, 100), clamp=True), title="Mort Rate Per 100k"),
    tooltip=["Rate"],
).properties(
    title=f"{cancer} mortality rates for {'males' if sex == 'M' else 'females'} in {year}",
)